In [1]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 300)
pd.set_option('display.max_rows', 300)

In [2]:
import json

with open("../data/fix_ipc_class_dict.json") as json_file:
    ipc_class_dict = json.load(json_file)

In [3]:
#df = pd.read_csv("../data/subest1_filtrato.csv")
df = pd.read_csv("../data/data_subset1.csv")

In [4]:
df.title = df.title.str.lower()
df.abstract = df.abstract.str.lower()
df.claims = df.claims.str.lower()
df.ipc_classes = df.ipc_classes.apply(lambda x: x.replace(",", ", ") if type(x)==str else "")

In [5]:
df.head()

,filename,filing_date,publication_date,priority_date,ipc_classes,assignee,inventors,docdb_family_id,title,abstract,claims
0,US2009216698_A1.txt,2008-02-22,2009-08-27,2008-02-22,"G06F15/18, G06N5/02","XEROX CORPORATIONANDREOLI, JEAN-MARCBOUCHARD, GUILLAUME","ANDREOLI, JEAN-MARCBOUCHARD, GUILLAUME",40999265,temporal events analysis employing tree induction,an events analysis method comprises: optimizing respective to a set of training data a set of branching transition likelihood parameters associating parent events of type k with child events of type k' in branching processes; inferring a most probable branching process for a set of input data co...,1. an events analyzer comprising: a probabilistic branching process learning engine configured to optimize respective to a set of training data a set of branching transition likelihood parameters associating parent events of type k with child events of type k′ in branching processes; and a proba...
1,US2020018760_A1.txt,2019-09-23,2020-01-16,2007-03-27,G01N33/574,IMMUNOVIA,"BORREBAECK, CARL, ARNE, KRISTERWINGREN, LARS, BERTIL, CHRISTER",39493874,protein signature/markers for the detection of adenocarcinoma,the present invention provides a method for determining the presence of pancreatic adenocarcinoma in an individual and/or for determining the survival time of an individual afflicted with pancreatic adenocarcinoma comprising the steps of: (a) providing a serum or plasma sample to be tested; and ...,1. a method for determining the presence of pancreatic adenocarcinoma in an individual comprising the steps of: a) providing a serum or plasma sample to be tested; b) determining a protein signature of the test sample by measuring the presence and/or amount in the test sample of one or more prot...
2,EP3340117_A1.txt,2017-11-15,2018-06-27,2016-12-20,"G06N3/04, G06N3/063",INTEL CORPORATION,"LIN, TSUNG-HAN",60327233,unsupervised learning using neuromorphic computing,"a spiking neural network (snn) is implemented on a neuromorphic computers and includes a plurality of neurons, a first set of the plurality of synapses defining feed-forward connections from a first subset of the neurons to a second subset of the neurons, a second subset of the plurality of syna...","1. a method comprising: defining a spiking neural network (snn) comprising a plurality of artificial neurons interconnected by a plurality of artificial synapses, wherein the snn is to comprise: a first subset of the plurality of synapses to define feedforward connections from a first subset of ..."
3,WO2019070570_A1.txt,2018-10-01,2019-04-11,2017-10-06,A61B5/103,TELLUS YOU CARE,"HSU KEVINCOKE, TANIA",65993331,non-contact activity sensing network for elderly care,"determining a physical state of a person includes detecting positions of different portions of the person, transforming detected positions of the person into a point cloud having a density that varies according to movement of each of the portions, correlating movement and position data from the ...","1. a method of determining a physical state of a person, comprising:detecting positions of different portions of the person;transforming detected positions of the person into a point cloud having a density that varies according to movement of each of the portions;correlating movement and positio..."
4,US2016259635_A1.txt,2015-03-04,2016-09-08,2015-03-04,"G06F15/18, G06F17/27, G06F8/65, G06F8/71, G06F9/44, G06F9/445, G06N99/00",IBM (INTERNATIONAL BUSINESS MACHINES CORPORATION),"EL MAGHRAOUI, KAOUTARJANN, JOEFONPATTNAIK, PRATAP C.PICKOVER CLIFFORD A.",56849635,software patch management incorporating sentiment analysis,"a method and system are provided. the method includes generating, by a machine-based sentiment prediction generator, respective machine-determined sentiment predictions for each of a plurality of software patches using sentiment analysis. the method further includes setting, by a sentiment-based...","1. 1-18. (canceled)19. a computer program product for softwar

In [6]:
len(df)

216986

<h3>Keyword della professoressa:</h3>

<img src="keyword prof.png" style="width:800px;"/>

In [55]:
keywords_other = []
"""["molecule",
                  "particle", 
                  "pharmacological",
                  "technolog",
                  "nano(-| )?tech(nolog)?",
                  "alter(ing)?",
                  "nano",
                  "nano(-| )?bot",
                  "nano(-| )?emulsion",
                  "nano(-| )?particle",
                  "nano(-| )?tube",
                  "nano(-| )?sphere",
                  "quantum",
                  "graphene",
                  "cloud",
                  "software",
                  "product"
                 ]"""

"""
Possiamo cercare i vincoli espressi dalla professoressa sia in "stile pandas" 
    (cercare documenti che contengono sia "body" che "care": df[(df.body >= 1) & (df.care >= 1)])
    
    che tramite l'uso delle espressioni regolari per implmentare l'operatore che l'Ing Ghironi 
    ha chiamto NEAR. Ad esempio per cercare "skin NEAR care" possiamo aggiungere alla lista sotto
    "lip(\w|\W){1,20}?stick". Agli estremi ci sono le 2 parole in cui siamo interessati, 
    "(\w|\W){1,20}?" serve invece a dire "permetti la presenza tra 
    0 e 20 caratteri o numeri tra le 2 parole". Se volete fare dei test: https://pythex.org/
"""

keywords_cosmetics = ["cosmetic",
                      "beauty", 
                      "body", 
                      "skin", 
                      "nail", 
                      "fingernail", 
                      "cleans(e|ing)", # 
                      "manicur(e|ing)", #manicure and manicuring
                      "pedicur(e|ing)",
                      "varnish", 
                      "polish",
                      "nail(s| |-)*polish",
                      "lip(s)?",
                      "lip( |-)?stic( |k)+",
                      "blush",
                      "mascara",
                      "make(s| |-)*up",#makes up, make-up
                      "cream",
                      "soap",
                      "toothpaste",
                      "sunscreen",
                      "lotion",
                      "deodorant",
                      "care",
                      "lip(s| |-)*(care|polish)",
                      "skin(s| |-)*(care|polish)",
                      "skin(s| |-)*(care|polish)",
                      "nail(s| |-)*(care|polish)",
                      "baby(s| |-)*(care|polish)",
                      "anti(-)?ag(e|ing)",
                      "colour|color",
                      "finger"
                     ]

In [8]:
df_original = df.copy()

### Controllo IPC (DA AFFINARE)
##### IPC A45D+

HAIRDRESSING OR SHAVING EQUIPMENT ; EQUIPMENT FOR COSMETICS OR COSMETIC TREATMENTS, e.g. FOR MANICURING OR PEDICURING

In [9]:
df_selected = pd.DataFrame()

In [12]:
df_candidate = df[(df.ipc_classes.str.count("(^| )A45D")>0)
                 & (df.ipc_classes.str.count("(^| )G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("(^| )A61B5")==0) #	Measuring for diagnostic purposes
                 ]
df_candidate[["title", "ipc_classes","claims"]] 

,title,ipc_classes,claims
3240,system for dispensing at least one makeup product and method for dispensing and evaluating makeup,"A45D40/00, A45D40/26, A45D44/00, B01F13/10, B01F15/02, B01F5/06, G01J3/46","1. a system for dispensing at least one makeup product and that can be manipulated in one hand, the system comprising: a cup; and a dispenser for filling the cup from a bottom of the cup with at least two base products, wherein: the cup is secured to the dispenser at least while the cup is being..."
20938,automatic nail polish application system and method,"A45D29/00, A45D29/11, A45D29/12, A45D29/14, A45D34/04","1. a system for automatically polishing a nail of a user including:an end-effector havinga cartridge receiving unit,a cartridge including nail polish, anda nozzle;a sensor, to generate target signals from a target location;a nail determination unit, receiving said target signals, to automaticall..."
31462,training system and device,"A45D1/00, G09B19/00, G09B19/24, G09B5/04","1. a beauty product system for performing a treatment on a user, the system comprising:an input for allowing the user to specify a desired treatment to be performed; a beauty product device that is manipulate by the user to perform the desired treatment on the user; the beauty product device hav..."
38540,cosmetic cleaning product,"A45D34/04, A61H15/02, A61H7/00, A61K8/00, A61K8/02, A61K8/31, A61K8/37, A61K8/42, A61K8/44, A61Q19/10, A61Q5/02","1. a cosmetic cleaning product, comprising: a massage applicator comprising a container body and a massage device having (a) at least one massage pin device, said pin device having at least one first surface which is configured to be in contact with the surface to be massaged, wherein a force ca..."
53114,portable apparatus for supporting a hand held hair dryer,"A45D20/12, A45D20/14, F16M11/00, F16M13/02","1. a portable hair dryer support apparatus, comprising: a hair dryer holding structure for holding a hair dryer; at least one vertical support element pivotally and directly connected at a bottom end to the hair dryer holding structure via a hinge having an axis of rotation perpendicular to the ..."
53136,improvements in and relating to hair irons,"A45D1/06, A45D1/28",1. an electric hair iron comprising at least one heating element and control means comprising a temperature sensor and a control circuit and wherein the control means is arranged such that power is only supplied to the heating element or elements when the temperature sensed by the sensor is at o...
61966,complete compact,"A45D33/00, A45D33/26, A45D44/00, G01N1/22, G01N1/24","i claim:1. a method for determining aroma comprising: (a) providing at least one means of aroma sensing capable of approximately discerning the presence or absence of at least one characteristic of the aroma, (b) sampling aroma with said aroma sensing means, (c) discerning said aroma characteris..."
63517,disposable single use applicator assembly with a chemical composition,"A45D37/00, A45D40/00, A45D40/26, A61K8/04, A61Q19/00","i claim:1. an assembly for storing and transporting, said assembly comprising: a wrapper having a top sheet and a bottom sheet, said top sheet being sealed to said bottom sheet so as to form a border along a perimeter of said top sheet and said bottom sheet; and an insert having a top surface an..."
69896,method for simulating the rendering of a make-up product on a body area,A45D44/00,"1. a method (200) for simulating a rendering of a makeup product on a body area, the method being performed by a processor (1 1 ), and comprising :o acquiring (210) an image of the body area without makeup of a subject, o determining (230) first color parameters of the pixels of the image corres..."
81748,system for targeted application of topical agents to an isolated body part,"A45D40/30, A45D44/00, A61K8/02, A61K8/368, A61P17/10, A61Q19/00, B32B38/10, B32B7/06, B33Y10/00, B33Y80/00, B44C3/02",1. a method for targeted application of topical agents to an isolated

In [13]:
df_selected = pd.concat([df_selected, df_candidate])

len(df_selected)

26

##### IPC C11B9/00 (riportato nelle slide come C11B/009)

In [14]:
df_candidate = df[(df.ipc_classes.str.count("(^| )C11B9/00")>0)
                 & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                 ]
df_candidate[["title", "ipc_classes","claims"]] 

,title,ipc_classes,claims
5337,method of making perfumed goods,"C11B9/00, C11D3/50","1. a method comprising: a.) using an odt value determined by mathematical modelling to select one or more perfume raw materials; b.) combining and/or processing said one or more perfume raw materials with one or more additional materials to form a product.2. the method of claim 1, wherein said p..."
135822,method of resolving allergens in perfume ingredients,"B01J20/26, B01J20/281, C11B9/00, G01N30/46, G01N30/86, G16C20/20","1. a method of resolving defined known or suspected allergens in a complex mixture of perfume ingredients using two-dimensional gas chromatography (2d-gc), wherein the combination of first and second stationary phases is selected on the basis of a clustering analysis of defined known or suspecte..."
174990,method of making perfumed goods,"A61L9/01, A61Q19/00, C11B9/00, C11D3/22, C11D3/50",what is claimed is:1. a method comprising:a.) using a malodour reduction value determined by mathematical modelling toselect one or more perfume raw materials;b.) combining and/or processing said one or more perfume raw materials with one or more additional materials to form a product. 2. the me...
210149,"methods for identifying, compounds identified and compositions thereof","A23L27/00, C11B9/00","1. a flavor/fragrance composition, comprising an effective amount of one or more compounds selected from tables 1-42.2. the flavor/fragrance composition of claim 1, wherein the composition is formulated as a spray, lotion, foam, gel, suspension, or emulsion.3. the flavor/fragrance composition of..."


In [15]:
df_selected = pd.concat([df_selected, df_candidate])

len(df_selected)

30

## Analisi Per keyword
#### Vedi slides Ing. Ghironi

In [56]:
for key in tqdm(keywords_cosmetics):
    df[key] = df.title.str.count(key).fillna(0) + df.abstract.str.count(key).fillna(0) + df.claims.str.count(key).fillna(0)
    
for key in tqdm(keywords_other):
    df[key] = df.title.str.count(key).fillna(0) #+ df.abstract.str.count(key).fillna(0) + df.claims.str.count(key).fillna(0)

  0%|          | 0/32 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [17]:
df["total_cosmetics"] = 0
for key in keywords_cosmetics:
    if key == "body" or key == "lip(s)?": continue
    df["total_cosmetics"] += df[key]
    
df["total_other"] = 0
for key in keywords_other:
    df["total_other"] += df[key]

In [18]:
keywords_cosmetics

['cosmetic',
 'beauty',
 'body',
 'skin',
 'nail',
 'fingernail',
 'cleans(e|ing)',
 'manicur(e|ing)',
 'pedicur(e|ing)varnish',
 'polish',
 'nail(s| |-)*polish',
 'lip(s)?',
 'lip( |-)?stic( |k)+',
 'blush',
 'mascara',
 'make(s| |-)*up',
 'cream',
 'soap',
 'toothpaste',
 'sunscreen',
 'lotion',
 'deodorant',
 'care',
 'lip(s| |-)*(care|polish)',
 'skin(s| |-)*(care|polish)',
 'skin(s| |-)*(care|polish)',
 'nail(s| |-)*(care|polish)',
 'baby(s| |-)*(care|polish)',
 'anti(-)?ag(e|ing)',
 'colour|color',
 'finger']

##### cosmetic

In [19]:
keyword = keywords_cosmetics[0]
keyword

'cosmetic'

In [20]:
df[df[keyword] > 0][["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims
1827,in vitro toxicogenomics for toxicity prediction using probabilistic component modeling and a compound-induced transcriptional response pattern,"C12Q1/68, C12Q1/6876, G06F19/12, G06F19/20, G16B20/00, G16B25/00, G16B5/00, G16C20/10, G16C20/30, G16C20/70","a novel method to predict toxicity and dose-dependent effects of an agent based on transcriptomic data analysis, by determining a predictive toxicogenomics space (ptgs) score. the ptgs score helps to predict and model the toxicity of compounds typically consisting of chemicals, pharmaceuticals, ...","1. a method of defining a predictive toxicogenomics space, ptgs, score indicative of a toxicity prediction for an agent, said method comprising: transforming gene data defining, for each instance of a plurality of instances, differential gene expression of genes in a test biological sample induc..."
3240,system for dispensing at least one makeup product and method for dispensing and evaluating makeup,"A45D40/00, A45D40/26, A45D44/00, B01F13/10, B01F15/02, B01F5/06, G01J3/46","a system for dispensing at least one makeup product, including a cup and a dispenser for filling the cup from the bottom with at least one product, the cup being secured to the dispenser at least while it is being filled with said product. the method for dispensing and evaluating makeup, includi...","1. a system for dispensing at least one makeup product and that can be manipulated in one hand, the system comprising: a cup; and a dispenser for filling the cup from a bottom of the cup with at least two base products, wherein: the cup is secured to the dispenser at least while the cup is being..."
5009,automated characterization and classification of microoganisms,"G01N21/65, G01N33/569","the present invention relates to an instrument, method, use and software program to obtain information rapidly about microorganisms that may spread uncontrolled in hospitals, water supply, food or when used in bio terrorism. vibrational spectroscopy provides data to a computer linked to one or m...","1. an instrument comprising a vibrational spectrometer, a processing means, a first spectral database [10] and a first information database [200], for obtaining information about a microorganism in a sample of interest by establishing vibrational spectroscopic similarity of the microorganism in ..."
6806,methods and systems for dynamic content provisioning,"G06F16/435, G06F16/438, G06F16/483, G06Q50/00","a method, system, and computer program product for dynamic content provisioning, such as for online marketing and product promotion, may include collecting user content from sources based upon collection parameters, and organizing the collected user content for optimization. there may also be th...","1. a system for dynamic content provisioning, comprising: at least one non-transitory memory configured to store computer program code instructions; and at least one processor configured to execute the computer program code instructions to: collect user content from one or more sources based on ..."
7363,personal appliance,"B26B19/38, B26B21/40",a personal appliance. the personal appliance includes a handle having an implement connecting structure. an implement is connected to the implement connecting structure. an implement displacement sensor is positioned in the handle. the implement displacement sensor measures a displacement of the...,"what is claimed is:1. a personal appliance comprising:a. a handle comprising an implement connecting structure;b. an implement connected to the implement connecting structure;c. an implement displacement sensor positioned in the handle, the implement displacement sensor measuring a displacement ..."
7691,systems and methods for measuring and categorizing colors and spectra of surfaces,"A45D44/00, A61B5/00, G01J3/02, G01J3/46","a color categorization system and method is described. the method comprises the steps of taking at least one color or spectral measurem

In [26]:
#escludiamo tutti i sistemi per "raccomandare un prodotto"

df_candidate = df[(df[keyword] > 0) 
                  & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G01")==0) #Escludo "MEASURING; TESTING"
                  & (df.ipc_classes.str.count("A61")==0) #MEDICAL OR VETERINARY SCIENCE; HYGIENE
                  & (df.ipc_classes.str.count("A01N")==0) #SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS, DETAILS, OR ACCESSORIES OF AGRICULTURAL MACHINES OR IMPLEMENTS, IN GENERAL
                  & (df.ipc_classes.str.count("(^| )A")>0)].query("title not in @df_selected")

df_candidate[["filename", "title", "ipc_classes","claims"]] 

,filename,title,ipc_classes,claims
27784,EP3733278_A1.txt,"moisture control device, moisture control method, program, storage medium, generated substance, product, device and equipment","A23L3/26, A23L3/30, A23L3/32, A23L5/30, A47J37/12, B01J19/08, B01J19/10, B01J19/12","1. a moisture control apparatus, wherein a predetermined voltage or current including a dc component and/or an ac component is applied to at least one electrode that generates at least one of an electric field, a magnetic field, an electromagnetic field, electromagnetic waves, sound waves, and u..."
106389,US2012003728_A1.txt,scalable portable sensory and yield expert system for biomass monitoring and production,"A01G31/02, C12M1/34, F21V33/00, G02F1/01","1. a system for optimizing plant growth in a liquid environment within an enclosed growth chamber, comprising: illumination means for controlling spatial, temporal and spectral characteristics of illumination surrounding the growth chamber, said illumination means comprising an array of individu..."
120991,EP3769852_A1.txt,"pump- actuated dispenser with integrated switch, system and method for remote monitoring of product consumption from said pump- actuated dispenser","A47K5/12, B05B11/00, B05B12/00, B05B15/30, H02N2/18","1. a pump-actuated dispenser (1) comprising: a compression pump (2) and a pump actuator (3) to operate the compression pump (2), the pump actuator (3) having a nozzle (4) for the output of a product to be dispensed, wherein the pump actuator (3) is displaceable between two end positions, a resti..."
166422,US2020405037_A1.txt,cosmetic package including a liquid reservoir,A45D34/04,"the embodiments of the invention in which an exclusive property or privilege is claimed are defined as follows:1. a cosmetic formula container, comprising: a container body configured to retain a portion of a cosmetic formula within a cavity, the container body having a neck configured to receiv..."
182147,US2020329742_A1.txt,"moisture control apparatus, moisture control method, program, storage medium, produced object, product, apparatus, and facility","A23L3/26, A23L3/30, A23L3/32, A23L5/10, A23L5/30, A47J37/12, B01J19/08, B01J19/10, B01J19/12, C02F1/48","1. a moisture control or current comprising: at least one electrode configured to generate at least one of an electric field, a magnetic field, an electromagnetic field, electromagnetic waves, sound waves, and ultrasonic waves, wherein a predetermined voltage or current including a dc component ..."
210149,US2020399558_A1.txt,"methods for identifying, compounds identified and compositions thereof","A23L27/00, C11B9/00","1. a flavor/fragrance composition, comprising an effective amount of one or more compounds selected from tables 1-42.2. the flavor/fragrance composition of claim 1, wherein the composition is formulated as a spray, lotion, foam, gel, suspension, or emulsion.3. the flavor/fragrance composition of..."


In [27]:
df_selected = pd.concat([df_selected, df_candidate])
len(df_selected)

36

##### beauty

In [28]:
keyword = keywords_cosmetics[1]
keyword

'beauty'

In [32]:
df_candidate = df[(df[keyword] > 0)
                  & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G01")==0) #Escludo "MEASURING; TESTING"
                  & (df.ipc_classes.str.count("A61")==0) #MEDICAL OR VETERINARY SCIENCE; HYGIENE
                  & (df.ipc_classes.str.count("C12Q")==0) #MEASURING OR TESTING PROCESSES INVOLVING ENZYMES, NUCLEIC ACIDS ...
                  & (df.ipc_classes.str.count("H04")==0) #ELECTRIC COMMUNICATION TECHNIQUE
                 ].query("title not in @df_selected")

df_candidate[["filename", "title", "ipc_classes","abstract", "claims"]]

,filename,title,ipc_classes,abstract,claims
5337,US2016369204_A1.txt,method of making perfumed goods,"C11B9/00, C11D3/50",the present invention relates to methods of designing and making perfumed products and perfume raw materials for use in products and perfume raw materials selected by such methods and the use of same.,"1. a method comprising: a.) using an odt value determined by mathematical modelling to select one or more perfume raw materials; b.) combining and/or processing said one or more perfume raw materials with one or more additional materials to form a product.2. the method of claim 1, wherein said p..."
7363,WO2019143922_A1.txt,personal appliance,"B26B19/38, B26B21/40",a personal appliance. the personal appliance includes a handle having an implement connecting structure. an implement is connected to the implement connecting structure. an implement displacement sensor is positioned in the handle. the implement displacement sensor measures a displacement of the...,"what is claimed is:1. a personal appliance comprising:a. a handle comprising an implement connecting structure;b. an implement connected to the implement connecting structure;c. an implement displacement sensor positioned in the handle, the implement displacement sensor measuring a displacement ..."
27784,EP3733278_A1.txt,"moisture control device, moisture control method, program, storage medium, generated substance, product, device and equipment","A23L3/26, A23L3/30, A23L3/32, A23L5/30, A47J37/12, B01J19/08, B01J19/10, B01J19/12","provided are a moisture control apparatus, a moisture control method, a program, a storage medium, a produced object, a product, an apparatus, and a facility that can improve the characteristics of an object through moisture control. in a moisture control apparatus according to one aspect of the...","1. a moisture control apparatus, wherein a predetermined voltage or current including a dc component and/or an ac component is applied to at least one electrode that generates at least one of an electric field, a magnetic field, an electromagnetic field, electromagnetic waves, sound waves, and u..."
31462,WO2019122838_A1.txt,training system and device,"A45D1/00, G09B19/00, G09B19/24, G09B5/04",a beauty product device is disclosed which includes a variety of sensors for sensing the way in which a user is operating the device. the sensor data can be processed and compared with pre-stored data representative of a desired style and feedback messages generated and provided back to the user...,"1. a beauty product system for performing a treatment on a user, the system comprising:an input for allowing the user to specify a desired treatment to be performed; a beauty product device that is manipulate by the user to perform the desired treatment on the user; the beauty product device hav..."
62406,US2019224871_A1.txt,personal appliance,"B26B19/38, B26B21/40, B26B21/52",a personal appliance. the personal appliance includes a handle having an implement connecting structure. an implement is connected to the implement connecting structure. an implement displacement sensor is associated with the handle. the implement displacement sensor measures a displacement of t...,"1. a personal appliance comprising: a. a handle comprising an implement connecting structure; b. an implement connected to the implement connecting structure; c. an implement displacement sensor associated with the handle, the implement displacement sensor measuring a displacement of the impleme..."
140736,US2020171687_A1.txt,personal appliance,"B26B19/38, B26B21/40",a personal appliance. the personal appliance includes a handle having an implement connecting structure. an implement is connected to the implement connecting structure. an implement displacement sensor is positioned in the handle. the implement displacement sensor measures a displacement of the...,"1. a personal appliance comprising: a. a handle comprising an implement connecting structure; b. an implement connect

In [33]:
df_selected = pd.concat([df_selected, df_candidate])
len(df_selected)

43

##### body (troppo rumore)

In [34]:
keyword = keywords_cosmetics[2]
keyword

'body'

In [35]:
df_candidate = df[(df[keyword] > 0)
                  & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G01")==0) #Escludo "MEASURING; TESTING"
                  & (df.ipc_classes.str.count("A61")==0) #MEDICAL OR VETERINARY SCIENCE; HYGIENE
                  & (df.ipc_classes.str.count("A63H")==0) #TOYS, e.g. TOPS, DOLLS, HOOPS OR BUILDING BLOCKS
                  & (df.ipc_classes.str.count("G09")==0) #EDUCATION; CRYPTOGRAPHY; DISPLAY; ADVERTISING; SEALS
                  & (df.ipc_classes.str.count("G10")==0) #MUSICAL INSTRUMENTS; ACOUSTICS
                  & (df.ipc_classes.str.count("(^| )A")>0)
                  ].query("title not in @df_selected")[["title", "ipc_classes","abstract", "claims"]]

df_candidate

,title,ipc_classes,abstract,claims
566,systems and methods for data-driven movement skill training,"A63B24/00, A63B69/40, A63B71/06",a data-driven movement skill training system is disclosed. the system uses movement skill assessment and diagnostics at distinct levels of the human movement system hierarchy to specify training goals for a user. the system may provide various augmentations that are synthesized to help the user ...,"1. an apparatus for movement skill training, the apparatus comprising: a sensor system comprising one or more sensors configured to obtain movement data for a subject performing an activity; a processor system in communication with the one or more sensors, the processor system having a microproc..."
2654,mobile robot and control method therefor,"A47L9/28, B25J11/00, B25J19/02, B25J9/00, B25J9/16, G05D1/02",a mobile robot in one general aspect of the present invention includes: a travel drive unit configured to move a main body; an image acquisition unit configured to acquire an image of surroundings of the main body; a sensor unit having one or more sensors configured to sense an object during the...,1. a mobile robot comprising: a travel drive unit configured to move a main body; an image acquisition unit configured to acquire an image of surroundings of the main body; a sensor unit having one or more sensors configured to sense an object during the movement of the main body; a storage conf...
3478,connected food preparation system and method of use,"A23L5/10, A47J36/32, A47J37/06, F24C15/00, F24C3/12, F24C7/08, H04W4/80, H05B1/02","a connected oven, including a set of in-cavity sensors and a processor configured to automatically identify foodstuff within the cooking cavity, based on the sensor measurements; and automatically operate the heating element based on the foodstuff identity.","we claim:1. a connected oven, comprising: an oven body defining a cooking cavity, the oven body comprising an oven base opposing an oven top, the cooking cavity comprising a set of low reflectance surfaces; a set of optical sensors cooperatively defining a field of view, the set of optical senso..."
4902,dribbling and ball control sports training system and method,A63B69/00,"a sports training method is described comprising the following steps: classifying the dribbling and ball or puck control motions into modular predetermined motion sequences according to the sensomotor and technical skills developed, evaluating athlete's skill level in relation to dribbling and b...","1. a sports training method which comprises the following steps: classifying different dribbling and ball or puck control motions into predetermined motion sequences according to the sensomotor and technical skills developed, evaluating athlete's skill level in relation to dribbling and ball ..."
5260,a connected oven,"A23L5/10, A47J36/32, A47J37/06, F24C15/00, F24C3/12, F24C7/06, F24C7/08, H04W4/80, H05B1/02, H05B6/68","a connected oven, including a set of in-cavity sensors and a processor configured to automatically identify foodstuff within the cooking cavity, based on the sensor measurements; and automatically operate the heating element based on the foodstuff identity.","1. a connected oven (100), comprising: • an oven body defining a cooking cavity (200), the oven body comprising an oven base (250) and an oven top (230); • an oven door (210) mounted to the oven body, the oven door (210) comprising: • a transparent window coextensive with the cooking cavity (200..."
8076,multiview body camera system with environmental sensors and alert features,"A41D1/00, A41D1/04, A42B3/04, G02B27/01, H04N5/00, H04N5/225, H04N5/247","a multiview body camera system with a vest, a helmet, shoulder pads, camera units providing video, directional microphones providing audio, sensors providing motion detection and other functions is described herein. a control module controls camera units, microphones, sensors and all other circu...",i claim:1. an enhanced situation

##### skin (da provare combinazioni)

In [36]:
keyword = keywords_cosmetics[3]
keyword

'skin'

In [37]:
df_candidate = df[(df[keyword] > 0)
                  & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G01")==0) #Escludo "MEASURING; TESTING"
                  & (df.ipc_classes.str.count("A61")==0) #MEDICAL OR VETERINARY SCIENCE; HYGIENE
                  & (df.ipc_classes.str.count("G09")==0) #EDUCATION; CRYPTOGRAPHY; DISPLAY; ADVERTISING; SEALS
                  & (df.ipc_classes.str.count("(^| )A")>0)
                 ].query("title not in @df_selected")

df_candidate[["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims
538,ligands for odor receptors and olfactory neurons,"A01M1/10, A01N33/00, A23L27/10",the disclosure provides compounds useful as insect repellents and compositions comprising such repellents. the disclosure further provides insect traps and method for identifying ligands and cognates for biological molecules.,"1-41. (canceled)42. a composition, comprising one or more compounds selected from table 9, table 10, and any combination thereof, in a trap, a cream, a lotion, a spray, a dust, a vapor emitter, a candle, an oil, a wicked apparatus, a fan, a vaporizer, a perfume, a cologne, a fragrance, a deodora..."
2533,rowing exercise machines having a configurable rowing feel,"A63B21/00, A63B21/005, A63B21/22, A63B22/00, A63B24/00, A63B71/06","among other things, a rowing exercise machine includes a movable inertial element, an eddy current brake coupled to the movable inertial element, a rowing grip coupled to the movable inertial element, and control circuitry coupled to the eddy current brake to cause a resistance to motion of the ...","1. a rowing exercise machine comprising a movable inertial element, an eddy current brake coupled to the movable inertial element, a rowing grip coupled to the movable inertial element, and control circuitry coupled to the eddy current brake to cause a resistance to motion of the rowing grip dur..."
4603,creating music by concatenative synthesis,"A63H5/00, G04B13/00, G10H7/00, H03G5/00","automated creation of new music by listening is disclosed. a method to create new music may comprise listening to a plurality of music, learning from the plurality of music, and performing concatenative synthesis based on the listening and the learning to create the new music. the method may be ...",it is claimed:1. a method to create new music comprising: listening to a plurality of music learning from the plurality of music performing concatenative synthesis based on the listening and the learning to create the new music.2. the method of claim 1 wherein the music comprises at least one of...
11876,personally powered appliance,"A43B3/00, G08B21/02, H01L41/113, H04B1/38, H04Q9/00","systems and methods are disclosed for powering a processor and a wireless transceiver in a sole having a sole plate and one or more spring elements disposed in a forefoot area of the sole plate, each of said spring elements including a piezoelectric generator coupled thereto, wherein each of the...","1. an apparatus, comprising: a processor; a wireless transceiver coupled to the processor; a power supply coupled to the processor and wireless transceiver located proximal to a sole, said sole including: a sole plate; and one or more leaf spring elements extending at an angle from the sole plat..."
12941,devices for prevention of pressure ulcers,"A47C27/00, A47C27/08","a support surface, such as an underpad, for preventing pressure sores in a patient is a multi-layer textile assembly having a skin contact layer formed from a fabric of synthetic filament yarns woven to provide air permeability and moisture vapor permeability and having a thickness of less than ...","1. a support surface for preventing pressure ulcers in a patient, the support surface comprising a multi-layer composite fabric comprising: a skin contact layer comprising a woven fabric of synthetic filament yarns woven to provide air permeability and moisture vapor permeability and having a th..."
15513,computer-implemented question and answer game,A63F9/24,"a computer-implemented question and answer game based on using opinion polls that have already been carried out on a representative sample of people. results of the opinion polls and predefined questions are stored in a knowledge base associated with the system, which is typically implemented on...","1. a method of allocating a prize to at least one individual using a computer system having at least one host computer terminal for presenting and collecting -data, a question engine for sendi

In [38]:
df_selected = pd.concat([df_selected, df[df.title == "personal care device with camera"]])
len(df_selected)

45

##### skin (da provare combinazioni)

In [39]:
keyword = keywords_cosmetics[4]
keyword

'nail'

In [40]:
df_candidate = df[(df[keyword] > 0)
                  & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G01")==0) #Escludo "MEASURING; TESTING"
                  & (df.ipc_classes.str.count("A61")==0) #MEDICAL OR VETERINARY SCIENCE; HYGIENE
                  & (df.ipc_classes.str.count("G16")==0)#NFORMATION AND COMMUNICATION TECHNOLOGY [ICT] SPECIALLY ADAPTED FOR SPECIFIC APPLICATION FIELDS
                  & (df.ipc_classes.str.count("G08")==0)#SIGNALLING
                  & (df.ipc_classes.str.count("G09")==0)#EDUCATION; CRYPTOGRAPHY; DISPLAY; ADVERTISING; SEALS
                  & (df.ipc_classes.str.count("H04")==0)#LECTRIC COMMUNICATION TECHNIQUE
                 ].query("title not in @df_selected")

df_candidate[["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims
10744,"improved microlayer membranes, mproved battery separators, and related methods","B29C48/00, B29C48/08, B29C48/21, B32B27/08, B32B27/32, H01M2/14, H01M2/16","described herein is a multilayer microporous film or membrane that may exhibit improved properties, including improved dielectric break down and strength, compared to prior monolayer or tri-layer microporous membranes of the same thickness. the preferred multilayer microporous membrane comprises...","1. a battery separator for a lithium battery comprising: at least one microporous separator membrane or sub-membrane comprising a plurality of porous or microporous polymer microlayers or nanolayers, wherein at least one of the individual microlayers or nanolayers comprises a different or distin..."
14944,power center assembly having electrical connection-protection and optional detachable surface mount,"H01R13/516, H01R13/66, H01R13/70, H01R25/00, H02G3/18","a power center assembly having a hooded member for protecting an electrical connection between a peripheral device plug and its respective outlet and a mount member for detachably mounting the power center to a surface. in particular, a power center assembly is provided having a member for prote...","1. a power center assembly, comprising: a power strip, said power strip having a housing member and at least one outlet for powering at least one peripheral device, said housing member having a bottom surface; and mechanical means for preventing inadvertent mechanical and electrical disconnec..."
20938,automatic nail polish application system and method,"A45D29/00, A45D29/11, A45D29/12, A45D29/14, A45D34/04",the present disclosure relates to a robotic apparatus and methods for automatic nail polish application on natural or artificial finger or toe nails. the robot uses artificial intelligence (al) to identify and paints the nails. the robot uses depth sensors and computer vision to plan the movemen...,"1. a system for automatically polishing a nail of a user including:an end-effector havinga cartridge receiving unit,a cartridge including nail polish, anda nozzle;a sensor, to generate target signals from a target location;a nail determination unit, receiving said target signals, to automaticall..."
22515,system and methods for patient data and treatment management,,"described are systems and methods for the analysis of information, particularly information obtained from the mass spectroscopic (proteomic) analysis of biological samples and associated patient and clinical information. the systems and methods of the invention may be used for a variety of purpo...","what is-1cealmeo fs:11. a system, comprising: a system input to receive a quantity of proteomic data obtained from mass spectrometric analysis of a biological sample; a metadata repository to store a quantity of metadata that includes the quantity of proteomic data; a rules database to store at ..."
27088,interchangeable footwear system and method,"A43B21/36, A43B3/24","a footwear system (s) and corresponding methods (m1) and (m2) for providing a plethora of footwear fashion options via a plurality of interchangeable components, the interchangeability of which being facilitated by structurally stable quick-coupling and quick-decoupling techniques. the system (s...","1. an interchangeable footwear system, comprising: a plurality of interchangeable uppers; a plurality of interchangeable heels; and a quick-release device comprising a rigid slide reversibly fastenable to the plurality of interchangeable uppers and reversibly fastenable to the plurality of inter..."
40165,"improved microlayer membranes, improved battery separators, and related methods","B29C47/06, B32B27/00, H01M10/052, H01M2/14, H01M2/16, H01M2/18","described herein is a multilayer microporous film or membrane that may exhibit improved properties, including improved dielectric break down and strength, compared to prior monolayer or tri-layer microporous membranes 

In [41]:
df_selected = pd.concat([df_selected, df[df.title == "automatic nail polish application system and method"]]) 
len(df_selected)

46

##### fingernail

In [42]:
keyword = keywords_cosmetics[5]
keyword

'fingernail'

In [43]:
df_candidate = df[(df[keyword] > 0)
                 ].query("title not in @df_selected")


df_candidate[["title", "ipc_classes","abstract", "claims"]] #Contiene solo metodi relativi al miglioramento dei dispositivi touch

,title,ipc_classes,abstract,claims
10595,method for improving accuracy of touch screen event analysis by use of spatiotemporal touch patterns,"G06F3/041, G06F3/0484, G06F3/0488","a method of classifying touch screen events uses known non-random patterns of touch events over short periods of time to increase the accuracy of analyzing such events. the method takes advantage of the fact that after one touch event, certain actions are more likely to follow than others. thus ...","1. a method of analyzing touch screen events based on characterization of features derived from each touch event, the method comprising: detecting a sequence of touch events on a touch sensitive surface (100); generating vibro-acoustic waveform signals using at least one sensor (110) associated ..."
23540,method for improving accuracy of touch screen event analysis by use of spatiotemporal touch patterns,"G06F3/041, G06F3/043","a method of classifying touch screen events uses known non-random patterns of touch events over short periods of time to increase the accuracy of analyzing such events. the method takes advantage of the fact that after one touch event, certain actions are more likely to follow than others. thus ...","1. a method of analyzing touch screen events based on characterization of features derived from each touch event, the method comprising: detecting touch events on a touch sensitive surface; generating vibro-acoustic waveform signals using at least one sensor associated with the touch events; con..."
25707,image processing using a convolutional neural network to track a plurality of objects,"G06K9/62, G06N3/04, G06N3/08, G06T11/00, G06T11/60, G06T3/40, G06T7/11, G06T7/194, G06T7/246, G06T7/73","presented is a convolutional neural network (cnn) model for fingernail tracking, and a method design for nail polish rendering. using current software and hardware, the cnn model and method to render nail polish runs in real-time on both ios and web platforms. a use of loss mean pooling (lmp) co...","we claim:1. a computing device comprising a processor and a storage device coupled thereto, the storage device storing a cnn and instructions, which when executed by the processor, configure the computing device to: process an image comprising a plurality of objects with the cnn, the cnn configu..."
35612,system and method for detecting imperfections in a screen,"G01N29/12, G01N29/46, G06F3/041, G06F3/0488","a method and system for detecting imperfections on a surface of a touchscreen of an electrical device, comprising: swiping a test object, such as a fingertip, a fingernail or a pin, along at least a portion of the touchscreen; producing, by the touchscreen, an electric signal indicative of the t...","1. a method of detecting imperfections on a surface of a touchscreen of an electrical device, comprising: swiping a test object along at least a portion of the touchscreen; producing, by the touchscreen, an electric signal indicative of the test object's contact with the touchscreen; receiving a..."
45418,image processing using a convolutional neural network to track a plurality of objects,"G06N3/02, G06N3/08, G06T1/40, G06T7/10, G06T7/194","presented is a convolutional neural network (cnn) model for fingernail tracking, and a method design for nail polish rendering. using current software and hardware, the cnn model and method to render nail polish runs in real-time on both ios and web platforms. a use of loss mean pooling (lmp) co...","w e claim:1. a computing device comprising a processor and a storage device coupled thereto, the storage device storing a cnn and instructions, which when executed by the processor, configure the computing device to: process an image comprising a plurality of objects with the cnn, the cnn config..."
52891,systems and methods for interactive virtual makeup experience,"G06F3/0484, G06F3/0488, G06T11/60, G06T7/136, G06T7/33, G06T7/60, G06T7/73, G06T7/90","a computing device for providing a virtual fingernail cosmetic ex

##### cleans(e|ing)

In [44]:
keyword = keywords_cosmetics[6]
keyword

'cleans(e|ing)'

In [48]:
df_candidate = df[(df[keyword] > 0)
                  & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G01")==0) #Escludo "MEASURING; TESTING"
                  & (df.ipc_classes.str.count("A61")==0) #MEDICAL OR VETERINARY SCIENCE; HYGIENE
                  & (df.ipc_classes.str.count("G16")==0)#NFORMATION AND COMMUNICATION TECHNOLOGY [ICT] SPECIALLY ADAPTED FOR SPECIFIC APPLICATION FIELDS
                  & (df.ipc_classes.str.count("G08")==0)#SIGNALLING
                  & (df.ipc_classes.str.count("G09")==0)#EDUCATION; CRYPTOGRAPHY; DISPLAY; ADVERTISING; SEALS
                  & (df.ipc_classes.str.count("H04")==0)#LECTRIC COMMUNICATION TECHNIQUE
                 ].query("title not in @df_selected")

df_candidate[["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims
136883,electric power grid supply and load prediction using cleansed time series data,"G05B15/02, H02J13/00, H02J3/00","an apparatus includes a processor to: assign each value of each set of values of an initial supply meter data and of an initial load meter data to one of multiple buckets based on weather conditions and/or time and date; for each bucket, generate upper and lower bounds of power provision and pow...","1. an apparatus comprising a processor and a storage to store instructions that, when executed by the processor, cause the processor to perform operations comprising: retrieve initial supply meter data from the storage, wherein: the initial supply meter data comprises, for each supply meter of m..."
142312,back-cleansing unit,"A46B13/04, A47K5/12, A47K7/02, A47K7/03, A47K7/04",an automatic back-cleansing apparatus for fixing to a vertical bath or shower surface includes a main housing section having a rotating head receiving section with gear teeth for engaging rotating heads on a rotating disk section adjacent the main housing section. the rotating disk section inclu...,"1. an automatic back-cleansing apparatus for fixation on a vertical surface, the apparatus comprising: a main housing section; a rotating head receiving section on the main housing section; wall mounting means on the main housing section; and a power producing means, wherein the rotating head re..."
155783,system and method for virtual energy assessment of facilities,"G05B13/02, G05F1/66","embodiments of the invention provide methods and systems to analyze energy consumption and support demand management of a portfolio of facilities. some embodiments of the invention include a computer implemented method for collecting and cleansing street addresses, time series energy consumption...","1. a computer-implemented system for remotely assessing energy performance of a plurality of facilities, the system comprising: a processor; a non-transitory computer-readable storage medium in data communication with the processor, the non-transitory computer-readable storage medium including s..."
169618,power meter based monitoring of elevator usage,B66B5/00,"the invention refers to a system comprising an elevator monitoring device (300) for monitoring at least one elevator (200) operatively coupled to at least one motor (240) which is operatively connected to at least one power meter (260), wherein the elevator monitoring device (300) is configured ...","1. a system (100) comprising an elevator monitoring device (300) for monitoring at least one elevator (200) operatively coupled to at least one motor (240) which is operatively connected to at least one power meter (260), wherein the elevator monitoring device (300) is configured to receive, fro..."
181560,power meter based monitoring of elevator usage,B66B5/00,"the invention refers to a system comprising an elevator monitoring device (300) for monitoring at least one elevator (200) operatively coupled to at least one motor (240) which is operatively connected to at least one power meter (260), wherein the elevator monitoring device (300) is configured ...","patent claims1. a system (100) comprising an elevator monitoring device (300) for monitoring at least one elevator (200) operatively coupled to at least one motor (240) which is operatively connected to at least one power meter (260), wherein the elevator monitoring device (300) is configured to..."
182424,apparatus and method for improved development of oil and gas wells,"E21B41/00, E21B44/00","a system, device and method are described for controlling development of an oil or gas well. data from the well is analysed to provide control signals for the well with the additional feature of cleaning the data in conjunction with the analysis. data is cleaned using knowledge based techniques,...","1. a system for controlling the development of an oil or gas well comprising: a data capture means for receiving well data; a data a

##### manicur(e|ing)

In [49]:
keyword = keywords_cosmetics[7]
keyword

'manicur(e|ing)'

In [51]:
df_candidate = df[(df[keyword] > 0)].query("title not in @df_selected")

df_candidate[["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims
20938,automatic nail polish application system and method,"A45D29/00, A45D29/11, A45D29/12, A45D29/14, A45D34/04",the present disclosure relates to a robotic apparatus and methods for automatic nail polish application on natural or artificial finger or toe nails. the robot uses artificial intelligence (al) to identify and paints the nails. the robot uses depth sensors and computer vision to plan the movemen...,"1. a system for automatically polishing a nail of a user including:an end-effector havinga cartridge receiving unit,a cartridge including nail polish, anda nozzle;a sensor, to generate target signals from a target location;a nail determination unit, receiving said target signals, to automaticall..."


In [52]:
df_selected = pd.concat([df_selected, df_candidate])

len(df_selected)

47

##### pedicur(e|ing)

In [57]:
keyword = keywords_cosmetics[8]
keyword

'pedicur(e|ing)'

In [58]:
df_candidate = df[(df[keyword] > 0)].query("title not in @df_selected")

df_candidate[["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims


In [63]:
keyword = keywords_cosmetics[11]
keyword

'nail(s| |-)*polish'

In [64]:
df_candidate = df[(df[keyword] > 0)].query("title not in @df_selected")

df_candidate[["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims
20938,automatic nail polish application system and method,"A45D29/00, A45D29/11, A45D29/12, A45D29/14, A45D34/04",the present disclosure relates to a robotic apparatus and methods for automatic nail polish application on natural or artificial finger or toe nails. the robot uses artificial intelligence (al) to identify and paints the nails. the robot uses depth sensors and computer vision to plan the movemen...,"1. a system for automatically polishing a nail of a user including:an end-effector havinga cartridge receiving unit,a cartridge including nail polish, anda nozzle;a sensor, to generate target signals from a target location;a nail determination unit, receiving said target signals, to automaticall..."
25707,image processing using a convolutional neural network to track a plurality of objects,"G06K9/62, G06N3/04, G06N3/08, G06T11/00, G06T11/60, G06T3/40, G06T7/11, G06T7/194, G06T7/246, G06T7/73","presented is a convolutional neural network (cnn) model for fingernail tracking, and a method design for nail polish rendering. using current software and hardware, the cnn model and method to render nail polish runs in real-time on both ios and web platforms. a use of loss mean pooling (lmp) co...","we claim:1. a computing device comprising a processor and a storage device coupled thereto, the storage device storing a cnn and instructions, which when executed by the processor, configure the computing device to: process an image comprising a plurality of objects with the cnn, the cnn configu..."
45418,image processing using a convolutional neural network to track a plurality of objects,"G06N3/02, G06N3/08, G06T1/40, G06T7/10, G06T7/194","presented is a convolutional neural network (cnn) model for fingernail tracking, and a method design for nail polish rendering. using current software and hardware, the cnn model and method to render nail polish runs in real-time on both ios and web platforms. a use of loss mean pooling (lmp) co...","w e claim:1. a computing device comprising a processor and a storage device coupled thereto, the storage device storing a cnn and instructions, which when executed by the processor, configure the computing device to: process an image comprising a plurality of objects with the cnn, the cnn config..."
52891,systems and methods for interactive virtual makeup experience,"G06F3/0484, G06F3/0488, G06T11/60, G06T7/136, G06T7/33, G06T7/60, G06T7/73, G06T7/90","a computing device for providing a virtual fingernail cosmetic experience generates an initial pattern located at a first position on a user interface displaying a digital image, the digital image further displaying a target object. the computing device obtains user input for relocating the init...","at least the following is claimed:1. a method implemented in a computing device having a processor, memory, and a display, the method for providing a virtual fingernail cosmetic experience, comprising: generating an initial pattern, the initial pattern being superimposed at a first position on a..."
69896,method for simulating the rendering of a make-up product on a body area,A45D44/00,"the invention proposes a method for simulating a rendering of a makeup product on a body area is disclosed comprising: - acquiring an image of the body area without makeup of a subject, - determining first color parameters of the pixels of the image corresponding to the body area without makeup,...","1. a method (200) for simulating a rendering of a makeup product on a body area, the method being performed by a processor (1 1 ), and comprising :o acquiring (210) an image of the body area without makeup of a subject, o determining (230) first color parameters of the pixels of the image corres..."
95446,method for providing a customized product recommendation,"G06F17/30, G06Q30/06",included is a method for providing a customized product recommendation to a user. images of people are collected from a database. a neural network is used to evalu

In [ ]:
##### VECCHIO CODICE:

In [49]:
df_selected[["filename"]].to_excel("selected_patents_filename_BIG.xlsx", index=False, header=False)

In [50]:
df_selected.to_excel("selected_patents_BIG.xlsx", index=False)

In [417]:
for ipc in very_interesting_ipc_classes:
    try:
        print(f"{ipc}: {ipc_class_dict[ipc].split(' - ')[-1]}")
    except:
        print(f"{ipc}: none")

A45D44/00: A45D44/00: Other cosmetic or personal care articles, e.g. for hairdressers' rooms + 
A45D29/14: A45D29/14: Manicuring or pedicuring implements + motor-driven
A45D29/00: A45D29/00: Manicuring or pedicuring + 
A61K8/00: A61K8/00: Cosmetics or similar toilet preparations + 
A61Q1/00: A61Q1/00: Make-up preparations; Body powders; Preparations for removing make-up + 
A61Q15/00A61Q19/10: none
C11B9/00: C11B9/00: Essential oils; Perfumes + 
A23L27/10: A23L27/10: Spices; Flavouring agents or condiments; Artificial sweetening agents; Table salts; Dietetic salt substitutes; Preparation or treatment thereof + Natural spices, flavouring agents or condiments; Extracts thereof
A45D34/04: A45D34/04: Containers or accessories specially adapted for handling liquid toilet or cosmetic substances, e.g. perfumes + Appliances specially adapted for applying liquid, e.g. using roller or ball
A45D31/00: A45D31/00: Artificial nails + 
A46B9/04: A46B9/04: Arrangements of the bristles in the brush body